<a href="https://colab.research.google.com/github/haaaram/MS-AIschool/blob/main/DeepPiCar(Project)/object_detection_ref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 1: Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_dir = '/content/drive/MyDrive/Colab Notebooks/TransferLearning/Training'
#!rm -rf '{model_dir}' ### model_dir 경로에 있는 파일 및 폴더 완전 제거
#os.makedirs(model_dir, exist_ok=True)
!ls -ltra '{model_dir}'/.. ### model_dir의 상위 디렉토리를 나열

## Section 2: Configs and Hyperparameters

In [ ]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/dctian/DeepPiCar' ### 레포 주소. 필요한 데이터와 모델을 다운로드한다.

# Number of training steps.(epoch 수)
num_steps = 1000  # 200000
#num_steps = 100  # 200000

# Number of evaluation steps.
num_eval_steps = 50


# model configs are from Model Zoo github:
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models
MODELS_CONFIG = { ### 사전 훈련된 객체 감지 모델에 대한 설정을 포함하는 딕셔너리
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
    'ssd_mobilenet_v1_quantized': {
        'model_name': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18',
        'pipeline_file': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config',
        'batch_size': 12
    },
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 12
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
# Note: for Edge TPU, you have to:
# 1) start with a pretrained model from model zoo, such as above 4
# 2) Must be a quantized model, which reduces the model size significantly
selected_model = 'ssd_mobilenet_v2_quantized'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Section 3: Set up Training Environment
Clone the DeepPiCar repository or your fork.

In [ ]:
import os

### 현재 위치는 /content
%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
'''
os.path.abspath(path)
 입력된 경로를 절대 경로(absolute path)로 변환한다.

os.path.join(path1, path2, ...)
 여러 개의 경로를 연결하여 하나의 경로를 생성한다.

os.path.basename(path)
 입력된 경로에서 파일 또는 디렉토리의 이름 부분을 추출한다.
'''

!git clone {repo_url}
%cd {repo_dir_path}

print('Pull it so that we have the latest code/data')
# pull -> 원격 저장소의 소스를 가져오고 해당 소스가 현재 내 소스보다 더 최신이면
# 지금의 버전을 해당 소스에 맞춰 올린다.
!git pull

Install required packages

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
# --quiet -> 에러 및 경고 메시지를 제외한 모든 메시지 출력x

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
'''
apt-get
  CLI 패키지 관리도구. 필요한 패키지를 설치한다.
  pip는 파이썬 패키지만 설치할 수 있는 반면, apt-get은 모든 종류의 패키지를 설치한다.

-qq
  조용히 설치

protpbuf-compiler
  Protocol Buffers 컴파일러.
  텐서플로우의 모델구조와 데이터를 직렬화 및 역직렬화하기 위해 사용된다.

python-pil
  Python Imaging Library(PIL) 또는 Pillow.
  이미지 처리와 조작을 위한 라이브러리이다.

python-lxml
  lxml은 XML 및 HTML 파싱 및 처리를 위한 Python 라이브러리이다.
  TensorFlow Object Detection API에서 주로 XML 파일을 다룰 때 사용한다.

python-tk
  Tkinter 라이브러리이다. GUI를 만들 때 사용된다.
  TensorFlow Object Detection API에서 이미지 시각화 및 편집 시 사용된다.
'''

!pip install -q Cython contextlib2 pillow lxml matplotlib
'''
Cython
  Python 프로그램을 C로 컴파일할 수 있는 도구.

contextlib2
  Python2에서 제공되는 contextlib 모듈을 Python3와 호환되도록 확장하는 라이브러리이다.
'''

!pip install -q pycocotools
# pycocotools -> COCO 데이터셋을 처리하고 분석하기 위한 도구 모음

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
'''
!protoc
  protocol buffer - 구글에서 개발한 직렬화 데이터 구조

--python_out=.
  현재 디렉토리('.')에 생성된 Python 파일을 저장한다.
'''

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py
# TensorFlow Object Detection API의 모델 빌더가 올바르게 구동되는지 확인

Prepare tfrecord files

In [ ]:
%cd {repo_dir_path}/models/object_detection

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python code/xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations
'''
-i : 입력 디렉토리 지정
-o : 출력 파일 경로 지정
-l : 레이블 디렉토리 지정
'''

# Convert test folder annotation xml files to a single csv.
!python code/xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python code/generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python code/generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

In [ ]:
test_record_fname = repo_dir_path + '/models/object_detection/data/annotations/test.record'
train_record_fname = repo_dir_path + '/models/object_detection/data/annotations/train.record'
label_map_pbtxt_fname = repo_dir_path + '/models/object_detection/data/annotations/label_map.pbtxt'

In [ ]:
!cat data/annotations/test_labels.csv
# 파일의 내용 출력
# cat -> concatenate, 파일을 연결하거나 파일의 내용을 출력할 때 사용

Download base model

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL_FILE = MODEL + '.tar.gz' # 'Tape Archive'로 묶인 후 'gzip' 알고리즘으로 압축됨
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    # urllib.request.urlretrieve -> 인터넷에서 파일을 다운로드한다.

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
    # 해당 디렉토리와 하위 디렉토리 및 파일을 모두 삭제
os.rename(MODEL, DEST_DIR)

In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}
'''
-a : 숨겨진 파일과 디렉토리도 포함하여 모든 파일과 디렉토리 표시
-ㅣ: 파일 및 디렉토리의 세부 정보(권한, 소유자, 크기 등)를 표시한다.
-h : 파일 크기를 인간이 읽기 쉬운 형식으로 표시한다.
'''

In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

## 4. Transfer Learning Training
Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
# assert: 파일이 존재하는지 확인
# 'assert 조건, 메시지': 주어진 조건을 확인하고, 조건이 False일 때 메시지가 출력된다.

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

# training pipeline file defines:
# - pretrain model path
# - the train/test sets
# - ID to Label mapping and number of classes
# - training batch size
# - epochs to trains
# - learning rate
# - etc

# note we just need to use a sample one, and make edits to it.

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:

    # fine_tune_checkpoint: downloaded pre-trained model checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # tfrecord files train and test, we created earlier with our training/test sets
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path: ID to label file
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps (Number of epochs to train)
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

'''
re.sub(pattern, replacement, string)
  pattern: 대체할 부분을 찾기 위한 정규 표현식 패턴
  replacement: 대체할 문자열
  string: 대체 작업을 수행할 대상 문자열
'''

In [ ]:
!cat {label_map_pbtxt_fname}

In [ ]:
# look for num_classes: 6, since we have 5 different road signs and 1 person type (total of 6 types)
!cat {pipeline_fname}

Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# wget -> 웹에서 파일을 다운로드한다.
!unzip -o ngrok-stable-linux-amd64.zip
# -o -> 덮어쓰기(overwrite).
#       zip 파일을 해제할 때 기존 파일을 덮어쓸지 묻지 않고 자동으로 덮어쓴다.

In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
'''
get_ipython().system_raw()
  Jupyter Notebook 또는 IPython 환경에서 명령줄 명령을 실행하는 데 사용

'tensorboard ...'
  TensorBoard를 실행한다.

--logdir "{}"
  로그 및 모델 파일의 디렉토리를 지정한다. LOG_DIR 내용이 {}부분에 들어간다

--host 0.0.0.0
  TensorBoard를 0.0.0.0 IP 주소로 실행하여 외부에서 접근할 수 있도록 한다.

--port 6006
  TensorBoard 웹 인터페이스를 위한 포트 번호를 지정한다.

& : 백그라운드에서 TensorBoard 실행한다.
'''

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')
# ngrok을 사용하여 로컬 포트 6006을 외부로 공개한다.

Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
'''
curl
  로컬 ngrok 서버에서 터널 정보를 가져온다. -s는 silent

|: 명령어를 연결함. 앞의 명령어의 출력을 뒤의 명령어의 입력으로 전달한다?

-c:  코드를 실행하도록 지시하는 옵션
'''

Train the model

In [ ]:
#################### SEND ALERT EMAIL AT FINISH WITH GMAIL #####################
# To send email from Python from your google account, MUST
# 1) Enable less secure app
# https://myaccount.google.com/lesssecureapps
# 2) Disable Unlock Capcha
# https://accounts.google.com/b/0/DisplayUnlockCaptcha

import smtplib

def SendEmail(msg):
    with open('/content/gdrive/My Drive/Colab Notebooks/pw.txt') as file:
        data = file.readlines()

    gmail_user = 'david.tian@gmail.com'
    gmail_password = data[0]


    sent_from = gmail_user
    to = ['dctian@hotmail.com']
    subject = msg
    body = '%s\n\n- David' % msg

    email_text = \
"""From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, email_text)
    server.quit()

    print(f'Email: \n{email_text}')

In [ ]:
num_steps = 2000
SendEmail("Colab train started")
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir='{model_dir}' \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}
# --alsologtostderr
#   로그 메시지가 터미널에 표시되고,
#   표준 출력('stdout')과 표준 오류('stderr')로 분리되지 않고 표시된다.
SendEmail("Colab train finished")

In [ ]:
!ls -ltra '{model_dir}'
'''
-ltra 옵션 -> l(long format), t(newest first), r(reverse order), a(all files)
'''

## Section 5: Save and Convert Model Output
Exporting a Trained Inference Graph

In [ ]:
import os
import re
import numpy as np

output_directory = '%s/fine_tuned_model' % model_dir
os.makedirs(output_directory, exist_ok=True)

In [ ]:
lst = os.listdir(model_dir)
# model_dir 안에 있는 파일과 서브 디렉토리의 목록을 가져옴

# find the last model checkpoint file, i.e. model.ckpt-1000.meta
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
# 'l'문자열에 'model.ckpt-'과 '.meta'가 포함되어 있는지
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])

last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

In [ ]:
!echo creates the frozen inference graph in fine_tune_model
# there is an "Incomplete shape" message.  but we can safely ignore that.
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory='{output_directory}' \
    --trained_checkpoint_prefix='{last_model_path}'

In [ ]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={pipeline_fname} \
    --trained_checkpoint_prefix='{last_model_path}' \
    --output_directory='{output_directory}' \
    --add_postprocessing_op=true

In [ ]:
!echo "CONVERTING frozen graph to quantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/road_signs_quantized.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

In [ ]:
!echo "CONVERTING frozen graph to unquantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/road_signs_float.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

In [ ]:
print(output_directory)
!ls -ltra '{output_directory}'
#pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb") # this is main one
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")  # this is tflite graph
!cp '{label_map_pbtxt_fname}' '{output_directory}'

Run inference test

In [ ]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname
print(PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "models/object_detection/data/images/test")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

In [ ]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [ ]:
# running inferences.  This should show images with bounding boxes
%matplotlib inline

print('Running inferences on %s' % TEST_IMAGE_PATHS)
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

Convert to Edge TPU's tflite Format

In [ ]:
# download this file from google drive.
!ls -lt '/content/gdrive/My Drive/Colab Notebooks/TransferLearning/Training/fine_tuned_model/road_signs_quantized.tflite'